# Row Filtering and Column Masking


In [0]:
%sql
drop schema if exists dkushari_uc.fgac cascade;
create schema if not exists dkushari_uc.fgac managed location "abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars/rlsclm/";

In [0]:
%sql
use catalog dkushari_uc;
use fgac;

In [0]:
%sql
select current_catalog();

current_catalog()
dkushari_uc


In [0]:
%sql
select current_schema();

current_schema()
fgac


In [0]:
%sql 
DROP TABLE IF EXISTS customer_pii_data_parquet;
DROP TABLE IF EXISTS customer_pii_data_delta;

In [0]:
from pyspark.sql.functions import expr, lit, rand
from pyspark.sql.types import IntegerType

# Generate a DataFrame with 50 records
df = spark.range(50).withColumnRenamed("id", "record_id")

# Add PII information (dummy data for illustration)
df = df.withColumn("first_name", expr("CASE WHEN rand() < 0.2 THEN 'John' " +
                                      "WHEN rand() < 0.4 THEN 'Jane' " +
                                      "WHEN rand() < 0.6 THEN 'Doe' " +
                                      "WHEN rand() < 0.8 THEN 'Alice' " +
                                      "ELSE 'Bob' END")) \
      .withColumn("last_name", expr("CASE WHEN rand() < 0.2 THEN 'Smith' " +
                                    "WHEN rand() < 0.4 THEN 'Johnson' " +
                                    "WHEN rand() < 0.6 THEN 'Williams' " +
                                    "WHEN rand() < 0.8 THEN 'Brown' " +
                                    "ELSE 'Jones' END")) \
      .withColumn("date_of_birth", expr("CASE WHEN rand() < 0.2 THEN '1980-01-01' " +
                                        "WHEN rand() < 0.4 THEN '1990-01-01' " +
                                        "WHEN rand() < 0.6 THEN '2000-01-01' " +
                                        "WHEN rand() < 0.8 THEN '1985-01-01' " +
                                        "ELSE '1995-01-01' END")) \
      .withColumn("age", (lit(2023) - expr("substring(date_of_birth, 1, 4)")).cast(IntegerType())) \
      .withColumn("gender", expr("CASE WHEN rand() < 0.5 THEN 'M' ELSE 'F' END")) \
      .withColumn("address", expr("CASE WHEN rand() < 0.2 THEN '123 Main St' " +
                                  "WHEN rand() < 0.4 THEN '456 Elm St' " +
                                  "WHEN rand() < 0.6 THEN '789 Pine St' " +
                                  "WHEN rand() < 0.8 THEN '101 Oak St' " +
                                  "ELSE '202 Maple St' END")) \
      .withColumn("ssn", expr("CASE WHEN rand() < 0.2 THEN '111-11-1111' " +
                              "WHEN rand() < 0.4 THEN '222-22-2222' " +
                              "WHEN rand() < 0.6 THEN '333-33-3333' " +
                              "WHEN rand() < 0.8 THEN '444-44-4444' " +
                              "ELSE '555-55-5555' END")) \
      .withColumn("region", expr("CASE WHEN rand() < 0.2 THEN 'Northeast' " +
                                 "WHEN rand() < 0.4 THEN 'Midwest' " +
                                 "WHEN rand() < 0.6 THEN 'South' " +
                                 "WHEN rand() < 0.8 THEN 'West' " +
                                 "ELSE 'Southwest' END"))

# Display the DataFrame
# display(df)

# Write the DataFrame to a Unity Catalog table
df.write.format("parquet").mode("overwrite").option("path","abfss://dkushari@dkusharistorageaccount.dfs.core.windows.net/mars/rlsclm/external-table").saveAsTable("customer_pii_data_parquet")

In [0]:
%sql
select * from dkushari_uc.fgac.customer_pii_data_parquet limit 10;

record_id,first_name,last_name,date_of_birth,age,gender,address,ssn,region
18,Doe,Williams,2000-01-01,23,F,123 Main St,111-11-1111,West
19,John,Brown,2000-01-01,23,M,456 Elm St,222-22-2222,Midwest
20,John,Williams,1990-01-01,33,F,789 Pine St,333-33-3333,Southwest
37,John,Brown,2000-01-01,23,M,789 Pine St,333-33-3333,Midwest
38,John,Smith,2000-01-01,23,M,789 Pine St,111-11-1111,Northeast
39,John,Brown,2000-01-01,23,F,789 Pine St,222-22-2222,Midwest
34,Jane,Brown,1995-01-01,28,F,123 Main St,222-22-2222,South
35,Jane,Brown,1990-01-01,33,M,789 Pine St,555-55-5555,Northeast
36,Doe,Brown,1985-01-01,38,F,456 Elm St,333-33-3333,Northeast
9,Jane,Johnson,1980-01-01,43,F,456 Elm St,555-55-5555,South


In [0]:
%sql
insert into dkushari_uc.fgac.customer_pii_data_parquet
select * from dkushari_uc.fgac.customer_pii_data_parquet limit 100;

In [0]:
%sql
insert into dkushari_uc.fgac.customer_pii_data_parquet
select * from dkushari_uc.fgac.customer_pii_data_parquet where first_name = 'Doe' and last_name = 'Williams'

In [0]:
%sql
select count(*) from dkushari_uc.fgac.customer_pii_data_parquet;

count(1)
106


In [0]:
%sql
create table if not exists customer_pii_data_delta deep clone customer_pii_data_parquet;

source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
52454,21,null,0,21,0,52454


## Customer Table with PII data

In [0]:
%sql
SELECT region, COUNT(*) AS total_customers
FROM customer_pii_data_parquet
GROUP BY region;

region,total_customers
Midwest,30
South,32
Southwest,4
West,20
Northeast,20


In [0]:
%sql
SELECT region, COUNT(*) AS total_customers
FROM customer_pii_data_delta
GROUP BY region;

region,total_customers
Midwest,30
South,32
Southwest,4
West,20
Northeast,20


# Create column masking UC functions 

In [0]:
%sql
use catalog dkushari_uc;
use fgac;

## The Column mask function does the folowing -
  - If the user is part of the mars_group_2 then they are not restricted to any masking
  - If the user is part of the mars_group_1 and satisfy certain criteria for another column then column_1 for those rows are not masked
  - Otherwise the data in column_1 is masked

In [0]:
%sql
drop function if exists fn_col_mask;
CREATE OR REPLACE FUNCTION fn_col_mask(column_1 STRING, column_2 STRING, column_value_3 STRING) 
  RETURN IF((is_account_group_member("mars_group_1") AND column_2 = column_value_3) OR 
            (is_account_group_member("mars_group_2")),
            column_1, "❌❌❌❌❌"
  );

## Alter table to add column mask - Using Delta table

In [0]:
%sql
ALTER TABLE customer_pii_data_delta ALTER COLUMN ssn DROP MASK;

#### For Gender=M, SSN is masked if the user belongs to mars_group_1

In [0]:
%sql
ALTER TABLE customer_pii_data_delta
  ALTER COLUMN ssn SET MASK fn_col_mask using columns (gender,'M');

### Test 1 - 
  - User is part of mars_group_2 which is allowed to see all the data as per the function

In [0]:
%sql
SELECT is_account_group_member('mars_group_1') as group_1, is_account_group_member('mars_group_2') as group_2;

group_1,group_2
false,true


In [0]:
%sql
select * from customer_pii_data_delta order by record_id limit 10;

record_id,first_name,last_name,date_of_birth,age,gender,address,ssn,region
0,Doe,Johnson,1985-01-01,38,F,101 Oak St,444-44-4444,South
0,Doe,Johnson,1985-01-01,38,F,101 Oak St,444-44-4444,South
1,Alice,Brown,2000-01-01,23,M,123 Main St,222-22-2222,Midwest
1,Alice,Brown,2000-01-01,23,M,123 Main St,222-22-2222,Midwest
2,Doe,Johnson,1985-01-01,38,F,789 Pine St,333-33-3333,Northeast
2,Doe,Johnson,1985-01-01,38,F,789 Pine St,333-33-3333,Northeast
3,Alice,Johnson,1990-01-01,33,M,202 Maple St,222-22-2222,Northeast
3,Alice,Johnson,1990-01-01,33,M,202 Maple St,222-22-2222,Northeast
4,Jane,Johnson,1985-01-01,38,F,456 Elm St,222-22-2222,Midwest
4,Jane,Johnson,1985-01-01,38,F,456 Elm St,222-22-2222,Midwest


### Test 2 - 
  - Group membership changed - User is now part of mars_group_1 which is Not allowed to see all the data as per the function
  - the association of the function to the table via Alter table only allows to see SSN data when gender is M

In [0]:
%sql
SELECT is_account_group_member('mars_group_1') as group_1, is_account_group_member('mars_group_2') as group_2;

group_1,group_2
true,false


In [0]:
%sql
select * from customer_pii_data_delta order by record_id limit 10;

record_id,first_name,last_name,date_of_birth,age,gender,address,ssn,region
0,Doe,Johnson,1985-01-01,38,F,101 Oak St,❌❌❌❌❌,South
0,Doe,Johnson,1985-01-01,38,F,101 Oak St,❌❌❌❌❌,South
1,Alice,Brown,2000-01-01,23,M,123 Main St,222-22-2222,Midwest
1,Alice,Brown,2000-01-01,23,M,123 Main St,222-22-2222,Midwest
2,Doe,Johnson,1985-01-01,38,F,789 Pine St,❌❌❌❌❌,Northeast
2,Doe,Johnson,1985-01-01,38,F,789 Pine St,❌❌❌❌❌,Northeast
3,Alice,Johnson,1990-01-01,33,M,202 Maple St,222-22-2222,Northeast
3,Alice,Johnson,1990-01-01,33,M,202 Maple St,222-22-2222,Northeast
4,Jane,Johnson,1985-01-01,38,F,456 Elm St,❌❌❌❌❌,Midwest
4,Jane,Johnson,1985-01-01,38,F,456 Elm St,❌❌❌❌❌,Midwest


## Alter table to add column mask - Using Parquet table

In [0]:
%sql
ALTER TABLE customer_pii_data_parquet ALTER COLUMN date_of_birth DROP MASK;
ALTER TABLE customer_pii_data_parquet ALTER COLUMN address DROP MASK;

#### Irrespective of Gender, DoB is masked if the user belongs to mars_group_1

In [0]:
%sql
ALTER TABLE customer_pii_data_parquet
  ALTER COLUMN date_of_birth SET MASK fn_col_mask USING COLUMNS (gender,'') ;

#### Address is masked for regions other than West when the user belongs to mars_group_1

In [0]:
%sql
ALTER TABLE customer_pii_data_parquet
  ALTER COLUMN address SET MASK fn_col_mask USING COLUMNS (region,'West') ;

### Test 3 - 
  - User is still part of mars_group_1 which restricts the user to see all the data as per the function definition
  - The user is not allowed is see any DoB data based on the first alter table statement
  - The user is only allowed to see address when region is West

In [0]:
%sql
SELECT is_account_group_member('mars_group_1') as group_1, is_account_group_member('mars_group_2') as group_2;

group_1,group_2
true,false


In [0]:
%sql
select * from customer_pii_data_parquet where region='West' limit 10;

record_id,first_name,last_name,date_of_birth,age,gender,address,ssn,region
32,Doe,Johnson,❌❌❌❌❌,33,M,456 Elm St,111-11-1111,West
18,Doe,Williams,❌❌❌❌❌,23,F,123 Main St,111-11-1111,West
16,Jane,Johnson,❌❌❌❌❌,38,F,789 Pine St,111-11-1111,West
25,Alice,Williams,❌❌❌❌❌,38,M,456 Elm St,222-22-2222,West
46,Jane,Smith,❌❌❌❌❌,23,F,789 Pine St,111-11-1111,West
18,Doe,Williams,❌❌❌❌❌,23,F,123 Main St,111-11-1111,West
46,Jane,Smith,❌❌❌❌❌,23,F,789 Pine St,111-11-1111,West
12,Alice,Williams,❌❌❌❌❌,43,F,456 Elm St,222-22-2222,West
14,Alice,Johnson,❌❌❌❌❌,33,F,101 Oak St,222-22-2222,West
16,Jane,Johnson,❌❌❌❌❌,38,F,789 Pine St,111-11-1111,West


In [0]:
%sql
select * from customer_pii_data_parquet limit 10;

record_id,first_name,last_name,date_of_birth,age,gender,address,ssn,region
46,Jane,Smith,❌❌❌❌❌,23,F,789 Pine St,111-11-1111,West
47,Doe,Smith,❌❌❌❌❌,38,F,❌❌❌❌❌,111-11-1111,South
48,Jane,Smith,❌❌❌❌❌,38,F,❌❌❌❌❌,222-22-2222,South
49,Jane,Brown,❌❌❌❌❌,43,M,❌❌❌❌❌,333-33-3333,Northeast
37,John,Brown,❌❌❌❌❌,23,M,❌❌❌❌❌,333-33-3333,Midwest
38,John,Smith,❌❌❌❌❌,23,M,❌❌❌❌❌,111-11-1111,Northeast
39,John,Brown,❌❌❌❌❌,23,F,❌❌❌❌❌,222-22-2222,Midwest
18,Doe,Williams,❌❌❌❌❌,23,F,123 Main St,111-11-1111,West
19,John,Brown,❌❌❌❌❌,23,M,❌❌❌❌❌,222-22-2222,Midwest
20,John,Williams,❌❌❌❌❌,33,F,❌❌❌❌❌,333-33-3333,Southwest


## Some more examples with new function definition

In [0]:
%sql
ALTER TABLE customer_pii_data_delta ALTER COLUMN ssn DROP MASK;
ALTER TABLE customer_pii_data_parquet ALTER COLUMN date_of_birth DROP MASK;
ALTER TABLE customer_pii_data_parquet ALTER COLUMN address DROP MASK;

In [0]:
%sql
drop function if exists fn_col_mask;
CREATE OR REPLACE FUNCTION fn_col_mask(column_1 STRING, column_2 STRING, column_value_2 STRING, column_3 STRING, column_value_3 STRING) 
  RETURN IF((is_account_group_member("mars_group_1") AND column_2 = column_value_2) OR
            (is_account_group_member("mars_group_2") AND column_3 = column_value_3),
            column_1, "❌❌❌❌❌"
  );

In [0]:
%sql
ALTER TABLE customer_pii_data_delta
  ALTER COLUMN ssn SET MASK fn_col_mask using columns (gender,'',region,'any');

In [0]:
%sql
ALTER TABLE customer_pii_data_parquet
  ALTER COLUMN date_of_birth SET MASK fn_col_mask using columns (region,'',gender,'');

In [0]:
%sql
SELECT is_account_group_member('mars_group_1') as group_1, is_account_group_member('mars_group_2') as group_2;

group_1,group_2
false,true


In [0]:
%sql
select * from customer_pii_data_delta limit 10;

record_id,first_name,last_name,date_of_birth,age,gender,address,ssn,region
18,Doe,Williams,2000-01-01,23,F,123 Main St,❌❌❌❌❌,West
19,John,Brown,2000-01-01,23,M,456 Elm St,❌❌❌❌❌,Midwest
20,John,Williams,1990-01-01,33,F,789 Pine St,❌❌❌❌❌,Southwest
37,John,Brown,2000-01-01,23,M,789 Pine St,❌❌❌❌❌,Midwest
38,John,Smith,2000-01-01,23,M,789 Pine St,❌❌❌❌❌,Northeast
39,John,Brown,2000-01-01,23,F,789 Pine St,❌❌❌❌❌,Midwest
34,Jane,Brown,1995-01-01,28,F,123 Main St,❌❌❌❌❌,South
35,Jane,Brown,1990-01-01,33,M,789 Pine St,❌❌❌❌❌,Northeast
36,Doe,Brown,1985-01-01,38,F,456 Elm St,❌❌❌❌❌,Northeast
9,Jane,Johnson,1980-01-01,43,F,456 Elm St,❌❌❌❌❌,South


In [0]:
%sql
select * from customer_pii_data_parquet limit 10;

record_id,first_name,last_name,date_of_birth,age,gender,address,ssn,region
18,Doe,Williams,❌❌❌❌❌,23,F,123 Main St,111-11-1111,West
19,John,Brown,❌❌❌❌❌,23,M,456 Elm St,222-22-2222,Midwest
20,John,Williams,❌❌❌❌❌,33,F,789 Pine St,333-33-3333,Southwest
37,John,Brown,❌❌❌❌❌,23,M,789 Pine St,333-33-3333,Midwest
38,John,Smith,❌❌❌❌❌,23,M,789 Pine St,111-11-1111,Northeast
39,John,Brown,❌❌❌❌❌,23,F,789 Pine St,222-22-2222,Midwest
34,Jane,Brown,❌❌❌❌❌,28,F,123 Main St,222-22-2222,South
35,Jane,Brown,❌❌❌❌❌,33,M,789 Pine St,555-55-5555,Northeast
36,Doe,Brown,❌❌❌❌❌,38,F,456 Elm St,333-33-3333,Northeast
9,Jane,Johnson,❌❌❌❌❌,43,F,456 Elm St,555-55-5555,South


# Appendix

In [0]:
%sql
drop function if exists fn_row_filter;
CREATE OR REPLACE FUNCTION fn_row_filter(column_value STRING)
  RETURN IF (is_account_group_member("ANALYST_SPAIN"), TRUE,  column_value="West"
  and EXISTS(SELECT 1 FROM valid_users v
    WHERE v.username = CURRENT_USER()));

In [0]:
%sql
ALTER TABLE customer_pii_data_delta
  SET ROW FILTER fn_row_filter ON (region);

In [0]:
%sql
ALTER TABLE customer_pii_data_parquet DROP ROW FILTER;


# UC Group membership lookup

In [0]:
%sql
SHOW GROUPS WITH USER `dipankar.kushari@databricks.com`;

name,directGroup
admins,true
dkushari-test-group-2025,true
team.fieldeng-all,true
ANALYST_USA,true
test_dk_group,true
mars_group_1,true
meijer_mkt_group,false
mygroup,true
all_analysts,false
meijer_sales_group,false


In [0]:
%sql
DROP TABLE IF EXISTS dkushari_uc.fgac.valid_users;

CREATE TABLE dkushari_uc.fgac.valid_users(username string);
INSERT INTO dkushari_uc.fgac.valid_users
VALUES
  ('dipankar.kushari@databricks.com'),
  ('sreeramreddy.thoom@databricks.com');


num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
SELECT is_account_group_member('ANALYST_SPAIN')

is_account_group_member(ANALYST_SPAIN)
false


In [0]:
%sql
drop table if exists valid_accounts;
CREATE TABLE if not exists valid_accounts(account string);
INSERT INTO valid_accounts
VALUES
  ('ANALYST_ES'),
  ('ANALYST_SPAIN');



num_affected_rows,num_inserted_rows
2,2
